# Business Case: Scaler Clustering

## Introduction
Scaler is an online tech-versity offering intensive computer science & Data Science courses through live classes delivered by tech leaders and subject matter experts. The meticulously structured program enhances the skills of software professionals by offering a modern curriculum with exposure to the latest technologies. It is a product by InterviewBit.

## Objective
We are provided with the information for a segment of learners and tasked to cluster them on the basis of their job profile, company, and other features. Ideally, these clusters should have similar characteristics.
The objective is to perform clustering on the dataset and come up with the best algorithm.

### Dataset profile
- **Email_hash**- Anonymised Personal Identifiable Information (PII)<br>
- **Company_hash** - This represents an anonymized identifier for the company, which is the current employer of the learner.<br>
- **orgyear**- Employment start date<br>
- **CTC**- Current CTC<br>
- **Job_position**- Job profile in the company<br>
- **CTC_updated_year**: Year in which CTC got updated (Yearly increments, Promotions)<br>

In [1]:
# -- importing libraries --
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [2]:
# -- importing dataset --
# the dataset has as unnamed column. we will remove it while reading the data
data = pd.read_csv('Data/scaler_clustering.csv').drop('Unnamed: 0', axis=1)
data.head()

,company_hash,email_hash,orgyear,ctc,job_position,ctc_updated_year
0,atrgxnnt xzaxv,6de0a4417d18ab14334c3f43397fc13b30c35149d70c05...,2016.0,1100000,Other,2020.0
1,qtrxvzwt xzegwgbb rxbxnta,b0aaf1ac138b53cb6e039ba2c3d6604a250d02d5145c10...,2018.0,449999,FullStack Engineer,2019.0
2,ojzwnvwnxw vx,4860c670bcd48fb96c02a4b0ae3608ae6fdd98176112e9...,2015.0,2000000,Backend Engineer,2020.0
3,ngpgutaxv,effdede7a2e7c2af664c8a31d9346385016128d66bbc58...,2017.0,700000,Backend Engineer,2019.0
4,qxen sqghu,6ff54e709262f55cb999a1c1db8436cb2055d8f79ab520...,2017.0,1400000,FullStack Engineer,2019.0


## Exploratory Data Analysis and Data Preprocessing

In [3]:
# info about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205843 entries, 0 to 205842
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   company_hash      205799 non-null  object 
 1   email_hash        205843 non-null  object 
 2   orgyear           205757 non-null  float64
 3   ctc               205843 non-null  int64  
 4   job_position      153279 non-null  object 
 5   ctc_updated_year  205843 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.4+ MB


In [4]:
# shape of the data
data.shape

(205843, 6)

In [5]:
# checking for missing values
data.isna().sum()

company_hash           44
email_hash              0
orgyear                86
ctc                     0
job_position        52564
ctc_updated_year        0
dtype: int64

In [6]:
# descriptive statistics
data.describe(include='all')

,company_hash,email_hash,orgyear,ctc,job_position,ctc_updated_year
count,205799,205843,205757.000000,2.058430e+05,153279,205843.000000
unique,37299,153443,NaN,NaN,1016,NaN
top,nvnv wgzohrnvzwj otqcxwto,bbace3cc586400bbc65765bc6a16b77d8913836cfc98b7...,NaN,NaN,Backend Engineer,NaN
freq,8337,10,NaN,NaN,43554,NaN
mean,NaN,NaN,2014.882750,2.271685e+06,NaN,2019.628231
std,NaN,NaN,63.571115,1.180091e+07,NaN,1.325104
min,NaN,NaN,0.000000,2.000000e+00,NaN,2015.000000
25%,NaN,NaN,2013.000000,5.300000e+05,NaN,2019.000000
50%,NaN,NaN,2016.000000,9.500000e+05,NaN,2020.000000
75%,NaN,NaN,2018.000000,1.700000e+06,NaN,2021.000000


#### Descriptive stats for the features

In [7]:
cat_cols = data.select_dtypes(include='object').columns
num_cols = data.select_dtypes(exclude='object').columns

In [8]:
# -- checking the distributions of the categorical variables --
for col in cat_cols:
    n = data[col].nunique()
    missing = data[col].isna().sum()
    mode = data[col].mode().values[0]
    mode_freq = data[col].value_counts().values[0]
    top_5 = data[col].value_counts().head(5)
    print(f'** Variable: {col} **')
    print(f'Number of unique values: {n}')
    print(f'Number of missing values: {missing}')
    print(f'Mode: {mode}')
    print(f'Mode frequency: {mode_freq}')
    print(f'Top 10 values: {top_5}')
    print('='*30)

** Variable: company_hash **
Number of unique values: 37299
Number of missing values: 44
Mode: nvnv wgzohrnvzwj otqcxwto
Mode frequency: 8337
Top 10 values: company_hash
nvnv wgzohrnvzwj otqcxwto    8337
xzegojo                      5381
vbvkgz                       3481
zgn vuurxwvmrt vwwghzn       3411
wgszxkvzn                    3240
Name: count, dtype: int64
** Variable: email_hash **
Number of unique values: 153443
Number of missing values: 0
Mode: bbace3cc586400bbc65765bc6a16b77d8913836cfc98b77c05488f02f5714a4b
Mode frequency: 10
Top 10 values: email_hash
bbace3cc586400bbc65765bc6a16b77d8913836cfc98b77c05488f02f5714a4b    10
6842660273f70e9aa239026ba33bfe82275d6ab0d20124021b952b5bc3d07e6c     9
298528ce3160cc761e4dc37a07337ee2e0589df251d73645aae209b010210eee     9
3e5e49daa5527a6d5a33599b238bf9bf31e85b9efa9a94f1c88c5e15a6f31378     9
b4d5afa09bec8689017d8b29701b80d664ca37b83cb883376b2e95191320da66     8
Name: count, dtype: int64
** Variable: job_position **
Number of unique valu

In [9]:
# -- checking the distributions of the categorical variables --
for col in num_cols:
    mean = data[col].mean()
    median = data[col].median()
    std = data[col].std()
    missing = data[col].isna().sum()
    min_val = data[col].min()
    max_val = data[col].max()
    q25 = data[col].quantile(0.25)
    q75 = data[col].quantile(0.75)
    print(f'** Variable: {col} **')
    print(f'Mean: {mean}')
    print(f'Standard Deviation: {std}')
    print(f'25th percentile: {q25}')
    print(f'Median: {median}')
    print(f'75th percentile: {q75}')
    print(f'Missing values: {missing}')
    print(f'Minimum value: {min_val}')
    print(f'Maximum value: {max_val}')
    print('='*30)


** Variable: orgyear **
Mean: 2014.8827500400957
Standard Deviation: 63.57111537643427
25th percentile: 2013.0
Median: 2016.0
75th percentile: 2018.0
Missing values: 86
Minimum value: 0.0
Maximum value: 20165.0
** Variable: ctc **
Mean: 2271685.0419931696
Standard Deviation: 11800914.44002465
25th percentile: 530000.0
Median: 950000.0
75th percentile: 1700000.0
Missing values: 0
Minimum value: 2
Maximum value: 1000150000
** Variable: ctc_updated_year **
Mean: 2019.6282312247683
Standard Deviation: 1.3251038701707907
25th percentile: 2019.0
Median: 2020.0
75th percentile: 2021.0
Missing values: 0
Minimum value: 2015.0
Maximum value: 2021.0


#### Treating missing values

In [10]:
# job position has a lot of missing values, but there is a category named 'Other' so we will replace the missing values with 'Other'
data['job_position'] = data['job_position'].fillna('Other').copy()

# since company hash has a low number of missing values, we will use the mode impution through sklearn simple imputer
mode_imputer = SimpleImputer(strategy='most_frequent')
data[['company_hash']] = mode_imputer.fit_transform(data[['company_hash']])

# org year also has a low number of missing values, so we will use the mode impution through sklearn knn imputer since it is a numeric value
knn_imputer = KNNImputer(n_neighbors=5)
data['orgyear'] = knn_imputer.fit_transform(data[['orgyear']])

#### Outlier removal

In [11]:
# removing the outliers using the IQR method with a threshold of 3
def remove_outliers(data, columns, threshold=1.5):
    n_rows_before = data.shape[0]
    print(f'Number of rows before removing outliers: {n_rows_before}')
    for col in columns:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - threshold*iqr
        upper_bound = q3 + threshold*iqr
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    n_rows_after = data.shape[0]
    print(f'Number of rows after removing outliers: {n_rows_after}')
    print(f'Percentage of rows removed: {((n_rows_before - n_rows_after)/n_rows_before)*100:.2f}%')
    return data
data = remove_outliers(data, num_cols, 3)

Number of rows before removing outliers: 205843
Number of rows after removing outliers: 199676
Percentage of rows removed: 3.00%


#### Feature engineering

In [15]:
# adding a new column for years of experience
data['years_of_experience'] = 2024 - data['orgyear']
# adding a new column for last incremental
data['years_since_last_increment'] = 2024 - data['ctc_updated_year']

# now we do not need the org year and ctc updated year columns
data = data.drop(['orgyear', 'ctc_updated_year'], axis=1)

In [16]:
data.head()

,company_hash,email_hash,ctc,job_position,years_of_experience,years_since_last_increment
0,atrgxnnt xzaxv,6de0a4417d18ab14334c3f43397fc13b30c35149d70c05...,1100000,Other,8.0,4.0
1,qtrxvzwt xzegwgbb rxbxnta,b0aaf1ac138b53cb6e039ba2c3d6604a250d02d5145c10...,449999,FullStack Engineer,6.0,5.0
2,ojzwnvwnxw vx,4860c670bcd48fb96c02a4b0ae3608ae6fdd98176112e9...,2000000,Backend Engineer,9.0,4.0
3,ngpgutaxv,effdede7a2e7c2af664c8a31d9346385016128d66bbc58...,700000,Backend Engineer,7.0,5.0
4,qxen sqghu,6ff54e709262f55cb999a1c1db8436cb2055d8f79ab520...,1400000,FullStack Engineer,7.0,5.0
